## Task B

Using the image URLs, obtain image labels (text) from Google Vision (cloud service)

In [1]:
import pandas as pd
import numpy as numpy
from google.cloud import vision
from google.oauth2 import service_account

In [27]:
#https://cloud.google.com/vision/docs/quickstart-client-libraries
def detect_labels_uri(uri) -> list:
    """Detects labels in the file located in Google Cloud Storage or on the
    Web."""
    credentials = service_account.Credentials.from_service_account_file('/home/jloesch30/.keys/ms3-unstruct-3159692ce6f1.json')

    client = vision.ImageAnnotatorClient()
    image = vision.Image()
    image.source.image_uri = uri

    response = client.label_detection(image=image)
    labels = response.label_annotations

    image_labels = [l.description for l in labels]
    return image_labels

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

In [3]:
posts_df = pd.read_csv("./posts_df.csv")

In [4]:
posts_df.head(2)

,id,likes,caption,imageURL
0,CWJOENGNxbJ,22066,Una splendida giornata #zarawoman,https://scontent-dfw5-1.cdninstagram.com/v/t51...
1,CWI6Mbjrxfi,27305,Una splendida giornata #zarawoman,https://scontent-dfw5-1.cdninstagram.com/v/t51...


In [28]:
# image url's are not persistant :(
# if you need to re-run this, you may need to scrape the images again due to the links having a timeout. You will recieve and error if
# if that is the case
all_labels = []
for i in posts_df['imageURL']:
    all_labels.append(detect_labels_uri(i))

In [29]:
# add the image labels to their respective image
posts_df['image_labels'] = all_labels

In [30]:
posts_df.head()

,id,likes,caption,imageURL,image_labels
0,CWJOENGNxbJ,22066,Una splendida giornata #zarawoman,https://scontent-dfw5-1.cdninstagram.com/v/t51...,"[Sky, Hat, People in nature, Happy, Sun hat, G..."
1,CWI6Mbjrxfi,27305,Una splendida giornata #zarawoman,https://scontent-dfw5-1.cdninstagram.com/v/t51...,"[Smile, Leg, Plant, Hat, Tree, People in natur..."
2,CWIkys4tab0,32700,Una splendida giornata #zarawoman,https://scontent-dfw5-1.cdninstagram.com/v/t51...,"[Smile, Sky, Happy, Flash photography, Travel,..."
3,CWEFjRns7OQ,16500,Zara SRPLS Fall ‘21. Drop 02 now available #za...,https://scontent-dfw5-1.cdninstagram.com/v/t51...,"[Clothing, Footwear, Joint, Outerwear, Leg, Hu..."
4,CWDpGDmJ5mV,21484,Zara SRPLS Fall ‘21. Drop 02 now available #za...,https://scontent-dfw5-1.cdninstagram.com/v/t51...,"[Forehead, Nose, Cheek, Chin, Eyebrow, Eyelash..."


In [32]:
posts_df.to_csv('posts_with_image_labels.csv', index=False)